In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import math
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yashpanjwani/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df = pd.read_csv('commentary_686.csv')

In [3]:
df

,srno,Data
0,0,8.55am Good morning and welcome to our coverag...
1,1,"Bracken to Smith, 1 run uppish and loose pushe..."
2,2,"Bracken to de Villiers, no run wide of off stu..."
3,3,"Bracken to de Villiers, no run again that swun..."
4,4,"Bracken to de Villiers, no run a similar line,..."
...,...,...
596,596,A clinical performance from the defending cham...
597,597,Australia win by seven wickets The South Afric...
598,598,"That's all from Jenny Thompson, Will Luke and ..."
599,599,Australia won by 7 wickets (with 111 balls rem...


In [4]:
corpus = ' '.join(df['Data'])

In [5]:
corpus

"8.55am Good morning and welcome to our coverage of the second semi-final between Australia and South Africa, with the winners taking on Sri Lanka in Saturday's final. Team and toss news to follow any second ... Bracken to Smith, 1 run uppish and loose pushed drive, in the air but into space, third man fields Bracken to de Villiers, no run wide of off stump, plenty of swing, aided by a stiff breeze, and de Villiers leaves Bracken to de Villiers, no run again that swung into the right-hander, de Villiers into line and drops into the leg side Bracken to de Villiers, no run a similar line, de Villiers pushes the ball no more than five or six yards into the leg side Bracken to de Villiers, no run outside off and wide enough to leave Bracken to de Villiers, FOUR runs de Villiers pushes forward, a thick edge shoots along the ground to the right of solitary slip and the ball flies across the outfield McGrath to Smith, 1 run Smith whips that from middle-and-leg into the on side, an easy run to

In [6]:
# def clean_text(corpus):
article=corpus.split(".")
article


['8',
 "55am Good morning and welcome to our coverage of the second semi-final between Australia and South Africa, with the winners taking on Sri Lanka in Saturday's final",
 ' Team and toss news to follow any second ',
 '',
 '',
 ' Bracken to Smith, 1 run uppish and loose pushed drive, in the air but into space, third man fields Bracken to de Villiers, no run wide of off stump, plenty of swing, aided by a stiff breeze, and de Villiers leaves Bracken to de Villiers, no run again that swung into the right-hander, de Villiers into line and drops into the leg side Bracken to de Villiers, no run a similar line, de Villiers pushes the ball no more than five or six yards into the leg side Bracken to de Villiers, no run outside off and wide enough to leave Bracken to de Villiers, FOUR runs de Villiers pushes forward, a thick edge shoots along the ground to the right of solitary slip and the ball flies across the outfield McGrath to Smith, 1 run Smith whips that from middle-and-leg into the on

In [7]:
sentences = []
for sentence in article :
    sentence = re.sub('[^a-zA-Z]',' ',str(sentence))
    sentence= re.sub(r'\s+',' ',sentence)
    sentences.append(sentence)
sentences.pop()
display = " ".join(sentences)
print('intial text: ')
print(display)
print('\n')
print(sentences)
filtered_list = [item for item in sentences if item.strip()]

# Print the filtered list
print(filtered_list)

intial text: 
   am Good morning and welcome to our coverage of the second semi final between Australia and South Africa with the winners taking on Sri Lanka in Saturday s final  Team and toss news to follow any second     Bracken to Smith run uppish and loose pushed drive in the air but into space third man fields Bracken to de Villiers no run wide of off stump plenty of swing aided by a stiff breeze and de Villiers leaves Bracken to de Villiers no run again that swung into the right hander de Villiers into line and drops into the leg side Bracken to de Villiers no run a similar line de Villiers pushes the ball no more than five or six yards into the leg side Bracken to de Villiers no run outside off and wide enough to leave Bracken to de Villiers FOUR runs de Villiers pushes forward a thick edge shoots along the ground to the right of solitary slip and the ball flies across the outfield McGrath to Smith run Smith whips that from middle and leg into the on side an easy run to the man 

In [8]:
def count(s):
    c=0
    wrds=word_tokenize(s)
    for word in wrds :
        c+=1
    return c
def cnt_in_sent(sentences):
    txt_data=[]
    i=0
    for s in sentences :
        i=i+1
        c = count(s)
        temp = {'id':i,'word_count':c}
        txt_data.append(temp)
    return txt_data



In [9]:
def freq_dict(sentences):
    freq_list = []
    for i, sent in enumerate(sentences, start=1):  # Using enumerate to get both the sentence and its index
        freq_dict = {}
        words = word_tokenize(sent)
        for word in words:
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1
        temp = {'id': i, 'word_count': freq_dict}
        freq_list.append(temp)
    return freq_list


In [10]:
def idf(text_data,freq_list):
    idfscores = []
    count = 0
    for item in freq_list:
        count = count +1
        for k in item['word_count']:
            val = sum([k in it['word_count'] for it in freq_list])
            temp = {
                'id' : count,
                'idfscore':math.log(len(text_data)/(val+1)),
                'key':k

            }
            idfscores.append(temp)
    return idfscores

In [11]:
#calculating tf-idf socres
def calctfidf(tf_scores,idfscores):
    tfidfscores = []
    for j  in idfscores:
        for i in tf_scores:
            if j['key']==i['key'] and j['id']==i['id']:
                temp={
                    'id':j['id'],
                    'tfidfscores':j['idfscore']*i['tf_score'],
                    'key':j['key']

                }
                tfidfscores.append(temp)
    return tfidfscores

In [12]:
def sent_scores(tfidf_scores,sentences,text_data):
    sent_data = []
    for txt in text_data:
        score = 0
        for i in range(0,len(tfidf_scores)):
            t_dict=tfidf_scores[i]
            if txt['id']==t_dict['id']:
                score=score+t_dict['tfidfscores']
        temp={
            'id':txt['id'],
            'score':score,
            'sentence':sentences[txt['id']-1]
        }
        sent_data.append(temp)
    return sent_data


In [13]:
def summary(sent_data):
    cnt = 0
    summary = []
    for t_dict in sent_data:
        cnt = cnt +t_dict['score']
    avg = cnt/len(sent_data)
    for sent in sent_data :
        if(sent['score']>=avg*0.9):
            summary.append(sent['sentence'])
    summary='.'.join (summary)
    return summary

In [14]:
text_data = cnt_in_sent(sentences)
text_data


[{'id': 1, 'word_count': 0},
 {'id': 2, 'word_count': 29},
 {'id': 3, 'word_count': 8},
 {'id': 4, 'word_count': 0},
 {'id': 5, 'word_count': 0},
 {'id': 6, 'word_count': 214},
 {'id': 7, 'word_count': 142},
 {'id': 8, 'word_count': 4},
 {'id': 9, 'word_count': 59},
 {'id': 10, 'word_count': 0},
 {'id': 11, 'word_count': 0},
 {'id': 12, 'word_count': 133},
 {'id': 13, 'word_count': 0},
 {'id': 14, 'word_count': 0},
 {'id': 15, 'word_count': 170},
 {'id': 16, 'word_count': 21},
 {'id': 17, 'word_count': 54},
 {'id': 18, 'word_count': 45},
 {'id': 19, 'word_count': 246},
 {'id': 20, 'word_count': 11},
 {'id': 21, 'word_count': 6},
 {'id': 22, 'word_count': 0},
 {'id': 23, 'word_count': 0},
 {'id': 24, 'word_count': 39},
 {'id': 25, 'word_count': 45},
 {'id': 26, 'word_count': 0},
 {'id': 27, 'word_count': 0},
 {'id': 28, 'word_count': 54},
 {'id': 29, 'word_count': 0},
 {'id': 30, 'word_count': 0},
 {'id': 31, 'word_count': 36},
 {'id': 32, 'word_count': 0},
 {'id': 33, 'word_count': 0},

In [15]:
freq_list = freq_dict(sentences)
freq_list

[{'id': 1, 'word_count': {}},
 {'id': 2,
  'word_count': {'am': 1,
   'Good': 1,
   'morning': 1,
   'and': 2,
   'welcome': 1,
   'to': 1,
   'our': 1,
   'coverage': 1,
   'of': 1,
   'the': 2,
   'second': 1,
   'semi': 1,
   'final': 2,
   'between': 1,
   'Australia': 1,
   'South': 1,
   'Africa': 1,
   'with': 1,
   'winners': 1,
   'taking': 1,
   'on': 1,
   'Sri': 1,
   'Lanka': 1,
   'in': 1,
   'Saturday': 1,
   's': 1}},
 {'id': 3,
  'word_count': {'Team': 1,
   'and': 1,
   'toss': 1,
   'news': 1,
   'to': 1,
   'follow': 1,
   'any': 1,
   'second': 1}},
 {'id': 4, 'word_count': {}},
 {'id': 5, 'word_count': {}},
 {'id': 6,
  'word_count': {'Bracken': 6,
   'to': 16,
   'Smith': 3,
   'run': 9,
   'uppish': 1,
   'and': 10,
   'loose': 1,
   'pushed': 1,
   'drive': 1,
   'in': 2,
   'the': 12,
   'air': 1,
   'but': 1,
   'into': 6,
   'space': 1,
   'third': 1,
   'man': 2,
   'fields': 1,
   'de': 14,
   'Villiers': 14,
   'no': 7,
   'wide': 5,
   'of': 4,
   'off':

In [16]:
def calc_TF(freq_list):
    tf_scores = []
    for item in freq_list:
        ID = item['id']
        word_count = sum(item['word_count'].values())  # Calculate total word count in the sentence
        for k in item['word_count']:
            tf_score = item['word_count'][k] / word_count  # Calculate TF score
            temp = {'id': ID, 'tf_score': tf_score, 'key': k}
            tf_scores.append(temp)
    return tf_scores

# Example usage


In [17]:
tfscore = calc_TF(freq_list)
tfscore

[{'id': 2, 'tf_score': 0.034482758620689655, 'key': 'am'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'Good'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'morning'},
 {'id': 2, 'tf_score': 0.06896551724137931, 'key': 'and'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'welcome'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'to'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'our'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'coverage'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'of'},
 {'id': 2, 'tf_score': 0.06896551724137931, 'key': 'the'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'second'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'semi'},
 {'id': 2, 'tf_score': 0.06896551724137931, 'key': 'final'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'between'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'Australia'},
 {'id': 2, 'tf_score': 0.034482758620689655, 'key': 'South'},
 {'id': 2, 'tf

In [18]:
idf_scores = idf(text_data,freq_list)
idf_scores

[{'id': 2, 'idfscore': 4.7318028369214575, 'key': 'am'},
 {'id': 2, 'idfscore': 3.8155121050473024, 'key': 'Good'},
 {'id': 2, 'idfscore': 4.7318028369214575, 'key': 'morning'},
 {'id': 2, 'idfscore': 0.7154198161690686, 'key': 'and'},
 {'id': 2, 'idfscore': 4.038655656361512, 'key': 'welcome'},
 {'id': 2, 'idfscore': 0.671359826375038, 'key': 'to'},
 {'id': 2, 'idfscore': 4.326337728813293, 'key': 'our'},
 {'id': 2, 'idfscore': 4.7318028369214575, 'key': 'coverage'},
 {'id': 2, 'idfscore': 1.297815632436311, 'key': 'of'},
 {'id': 2, 'idfscore': 0.671359826375038, 'key': 'the'},
 {'id': 2, 'idfscore': 3.3455084758015667, 'key': 'second'},
 {'id': 2, 'idfscore': 4.326337728813293, 'key': 'semi'},
 {'id': 2, 'idfscore': 3.3455084758015667, 'key': 'final'},
 {'id': 2, 'idfscore': 3.6331905482533475, 'key': 'between'},
 {'id': 2, 'idfscore': 3.122364924487357, 'key': 'Australia'},
 {'id': 2, 'idfscore': 3.3455084758015667, 'key': 'South'},
 {'id': 2, 'idfscore': 3.6331905482533475, 'key': 

In [19]:
tfidf = calctfidf(tfscore,idf_scores)
tfidf

[{'id': 2, 'tfidfscores': 0.16316561506625715, 'key': 'am'},
 {'id': 2, 'tfidfscores': 0.1315693829326656, 'key': 'Good'},
 {'id': 2, 'tfidfscores': 0.16316561506625715, 'key': 'morning'},
 {'id': 2, 'tfidfscores': 0.049339297666832316, 'key': 'and'},
 {'id': 2, 'tfidfscores': 0.13926398815039698, 'key': 'welcome'},
 {'id': 2, 'tfidfscores': 0.02315033884051855, 'key': 'to'},
 {'id': 2, 'tfidfscores': 0.1491840596142515, 'key': 'our'},
 {'id': 2, 'tfidfscores': 0.16316561506625715, 'key': 'coverage'},
 {'id': 2, 'tfidfscores': 0.044752263187459, 'key': 'of'},
 {'id': 2, 'tfidfscores': 0.0463006776810371, 'key': 'the'},
 {'id': 2, 'tfidfscores': 0.11536236123453679, 'key': 'second'},
 {'id': 2, 'tfidfscores': 0.1491840596142515, 'key': 'semi'},
 {'id': 2, 'tfidfscores': 0.23072472246907358, 'key': 'final'},
 {'id': 2, 'tfidfscores': 0.1252824326983913, 'key': 'between'},
 {'id': 2, 'tfidfscores': 0.10766775601680541, 'key': 'Australia'},
 {'id': 2, 'tfidfscores': 0.11536236123453679, 'k

In [20]:
sent_data = sent_scores(tfidf,sentences,text_data)
result = summary(sent_data)
print(result)

 am Good morning and welcome to our coverage of the second semi final between Australia and South Africa with the winners taking on Sri Lanka in Saturday s final. Team and toss news to follow any second . Bracken to Smith run uppish and loose pushed drive in the air but into space third man fields Bracken to de Villiers no run wide of off stump plenty of swing aided by a stiff breeze and de Villiers leaves Bracken to de Villiers no run again that swung into the right hander de Villiers into line and drops into the leg side Bracken to de Villiers no run a similar line de Villiers pushes the ball no more than five or six yards into the leg side Bracken to de Villiers no run outside off and wide enough to leave Bracken to de Villiers FOUR runs de Villiers pushes forward a thick edge shoots along the ground to the right of solitary slip and the ball flies across the outfield McGrath to Smith run Smith whips that from middle and leg into the on side an easy run to the man in the deep McGrat

In [21]:
df2 = df

In [22]:
import torch
from transformers import AutoTokenizer
# from scipy.special import softmax
from transformers import AutoModelForSequenceClassification

/Users/yashpanjwani/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
